In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras.backend as K
#tf.K.set_image_data_format(‘channels_last’)
tf.keras.backend.set_image_data_format('channels_last')
#import matplotlib.pyplot as plt
#from matplotlib.pyplot import imshow
path = 'G:\\TEXT\\jupyter\\Skin_attempt 4\\base_dir\\train_dir'
train_datagen = ImageDataGenerator(rescale=1. / 255)
train = train_datagen.flow_from_directory(path, target_size=(227,227), class_mode='categorical')

Found 25587 images belonging to 7 classes.


In [3]:
type(train)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [4]:
type(train_datagen)

tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [5]:
print("Batch Size for Input Image : ",train[0][0].shape)
print("Batch Size for Output Image : ",train[0][1].shape)
print("Image Size of first image : ",train[0][0][0].shape)
print("Output of first image : ",train[0][1][0].shape)

Batch Size for Input Image :  (32, 227, 227, 3)
Batch Size for Output Image :  (32, 7)
Image Size of first image :  (227, 227, 3)
Output of first image :  (7,)


In [6]:
def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
    X = Dense(7,activation='softmax',name = 'fc2')(X)
    
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    return model
alex = AlexNet(train[0][0].shape[1:])
alex.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv0 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 55, 55, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max0 (MaxPooling2D)          (None, 27, 27, 96)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 27, 27, 256)       1024

In [8]:
path1 = 'G:\\TEXT\\jupyter\\Skin_attempt 4\\base_dir\\test_dir\\val_dir'
val_datagen = ImageDataGenerator(rescale=1. / 255)
val = val_datagen.flow_from_directory(path1, target_size=(227,227), class_mode='categorical')
type(val)
type(val_datagen)

Found 2003 images belonging to 7 classes.


tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [10]:
alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])
alex.fit(train,epochs=5,validation_data=(val))

Epoch 1/5
800/800 [==============================] - 3338s 4s/step - loss: 1.1834 - accuracy: 0.5416 - val_loss: 1.1167 - val_accuracy: 0.5701
Epoch 2/5
800/800 [==============================] - 3326s 4s/step - loss: 1.0059 - accuracy: 0.6215 - val_loss: 0.9856 - val_accuracy: 0.6360
Epoch 3/5
800/800 [==============================] - 3326s 4s/step - loss: 0.8829 - accuracy: 0.6740 - val_loss: 1.1894 - val_accuracy: 0.5901
Epoch 4/5
800/800 [==============================] - 3358s 4s/step - loss: 0.7933 - accuracy: 0.7056 - val_loss: 1.0619 - val_accuracy: 0.6455
Epoch 5/5
800/800 [==============================] - 4025s 5s/step - loss: 0.7284 - accuracy: 0.7316 - val_loss: 0.8868 - val_accuracy: 0.6975


In [11]:
#testing
path_test = 'G:\\TEXT\\jupyter\\Skin_attempt 4\\base_dir\\test_dir\\testing_dir'
test_datagen = ImageDataGenerator(rescale=1. / 255)
test = test_datagen.flow_from_directory(path_test, target_size=(227,227), class_mode='categorical')

Found 2003 images belonging to 7 classes.


In [12]:
preds = alex.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))
alex.save("model1.h5")

63/63 [==============================] - 75s 1s/step - loss: 0.8503 - accuracy: 0.6945
Loss = 0.8503354787826538
Test Accuracy = 0.6944583058357239
